To get abodybuilder3
- Step 1: Clone repository
git clone https://github.com/Exscientia/abodybuilder3.git
cd abodybuilder3

- Step 2: Download model weights (small file)
mkdir -p output/ zenodo/
wget -P zenodo/ https://zenodo.org/records/11354577/files/output.tar.gz
tar -xzvf zenodo/output.tar.gz -C output/

- Step 3: Create conda environment
./init_conda_venv.sh

#- Step 4: Activate conda environment
conda activate ./.venv


---------------------------------------------------------------------------
ImportError                               Traceback (most recent call last)
File ~/abodybuilder3/.venv/lib/python3.9/site-packages/transformers/utils/import_utils.py:1382, in _LazyModule._get_module(self, module_name)
   1381 try:
-> 1382     return importlib.import_module("." + module_name, self.__name__)
   1383 except Exception as e:

File ~/abodybuilder3/.venv/lib/python3.9/importlib/__init__.py:127, in import_module(name, package)
    126         level += 1
--> 127 return _bootstrap._gcd_import(name[level:], package, level)

File <frozen importlib._bootstrap>:1030, in _gcd_import(name, package, level)
RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
cannot import name 'split_torch_state_dict_into_shards' from 'huggingface_hub' (/home/eva/abodybuilder3/.venv/lib/python3.9/site-packages/huggingface_hub/__init__.py)

In [3]:
!pip install --upgrade huggingface_hub


  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.20.1
    Uninstalling huggingface-hub-0.20.1:
      Successfully uninstalled huggingface-hub-0.20.1


In [4]:
!pip install "huggingface_hub>=0.23.0"

ProtT5 uses HuggingFace Transformers models under the hood (T5 backbone).

New Transformers need newer Huggingface Hub to deal with checkpoint sharding (split_torch_state_dict_into_shards).

Your abodybuilder3 installation is fine.
It’s just the surrounding transformers + huggingface_hub mismatch causing crash.

In [1]:
# ============================================================
# STEP 1B: Predict antibody structures with ABodyBuilder3 and ABodyBuilder3-LM
# ============================================================

import os
import pandas as pd
from tqdm import tqdm
import torch
from abodybuilder3.utils import string_to_input, output_to_pdb, add_atom37_to_output
from abodybuilder3.lightning_module import LitABB3
from abodybuilder3.language.model import ProtT5

# ============================================================
# --- Setup paths and basic parameters
# ============================================================

# Where to save your predictions
output_base = "/home/eva/Structure_model_benchmark/predictions_output"

# Location of your antibody CSV file (must have 'name', 'vh', 'vl' columns)
csv_path = "/home/eva/Structure_model_benchmark/antibody.csv"

# Location of your model checkpoint files
checkpoint_abodybuilder3 = "/home/eva/abodybuilder3/output/plddt-loss/best_second_stage.ckpt"
checkpoint_abodybuilder3_lm = "/home/eva/abodybuilder3/output/language-loss/best_second_stage.ckpt"

# Repeats - we predict 3x for each antibody
repeats = ["1", "2", "3"]

# Set device: use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ============================================================
# --- Create output folders
# ============================================================

for model_name in ["abodybuilder3", "abodybuilder3_lm"]:
    for rep in repeats:
        os.makedirs(os.path.join(output_base, f"{model_name}_{rep}"), exist_ok=True)

# ============================================================
# --- Load models
# ============================================================

# Load ABodyBuilder3 (normal)
module_ab3 = LitABB3.load_from_checkpoint(checkpoint_abodybuilder3)
model_ab3 = module_ab3.model
model_ab3.to(device)
model_ab3.eval()

# Load ABodyBuilder3-LM (language model version)
module_ab3_lm = LitABB3.load_from_checkpoint(checkpoint_abodybuilder3_lm)
model_ab3_lm = module_ab3_lm.model
model_ab3_lm.to(device)
model_ab3_lm.eval()

# Initialize ProtT5 model (for generating embeddings)
protT5 = ProtT5()

# ============================================================
# --- Load antibody sequences
# ============================================================

df = pd.read_csv(csv_path)

# ============================================================
# --- Define prediction functions
# ============================================================

def predict_abodybuilder3(name, vh, vl, out_path):
    """
    Predict structure using ABodyBuilder3 (no language model)
    """
    # Prepare input batch
    ab_input = string_to_input(heavy=vh, light=vl)
    ab_input_batch = {
        key: (value.unsqueeze(0).to(device) if key not in ["single", "pair"] else value.to(device))
        for key, value in ab_input.items()
    }

    # Forward pass
    with torch.no_grad():
        output = model_ab3(ab_input_batch, ab_input_batch["aatype"])

    # Postprocess and save PDB
    output = add_atom37_to_output(output, ab_input["aatype"].to(device))
    pdb_string = output_to_pdb(output, ab_input)
    with open(out_path, "w") as f:
        f.write(pdb_string)


def predict_abodybuilder3_lm(name, vh, vl, out_path):
    """
    Predict structure using ABodyBuilder3-LM (with ProtT5 language model)
    """
    # First, get ProtT5 embeddings
    with torch.no_grad():
        embedding = protT5.get_embeddings([vh], [vl]).to(device)

    # Prepare input batch
    ab_input = string_to_input(heavy=vh, light=vl)
    ab_input["single"] = embedding.unsqueeze(0)  # override single features with ProtT5
    ab_input_batch = {
        key: (value.unsqueeze(0).to(device) if key not in ["single", "pair"] else value.to(device))
        for key, value in ab_input.items()
    }

    # Forward pass
    with torch.no_grad():
        output = model_ab3_lm(ab_input_batch, ab_input_batch["aatype"])

    # Postprocess and save PDB
    output = add_atom37_to_output(output, ab_input["aatype"].to(device))
    pdb_string = output_to_pdb(output, ab_input)
    with open(out_path, "w") as f:
        f.write(pdb_string)

# ============================================================
# --- Run prediction loop
# ============================================================

for idx, row in tqdm(df.iterrows(), total=len(df)):
    name = row['name']
    vh_seq = row['vh']
    vl_seq = row['vl']

    # Predict with ABodyBuilder3
    for rep in repeats:
        out_path = os.path.join(output_base, f"abodybuilder3_{rep}", f"{name}_abodybuilder3_{rep}.pdb")
        try:
            predict_abodybuilder3(name, vh_seq, vl_seq, out_path)
        except Exception as e:
            print(f"Error predicting {name} (ABodyBuilder3 rep {rep}): {e}")

    # Predict with ABodyBuilder3-LM
    for rep in repeats:
        out_path = os.path.join(output_base, f"abodybuilder3_lm_{rep}", f"{name}_abodybuilder3_lm_{rep}.pdb")
        try:
            predict_abodybuilder3_lm(name, vh_seq, vl_seq, out_path)
        except Exception as e:
            print(f"Error predicting {name} (ABodyBuilder3-LM rep {rep}): {e}")

print("ABodyBuilder3 and ABodyBuilder3-LM prediction (3x repeats each) complete.")


/home/eva/abodybuilder3/.venv/lib/python3.9/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


: 